# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [15]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps.datasets
import gmaps
import os
import json

# Import API key
from api_keys import g_key
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
world_cities=pd.read_csv("../WeatherPy/Data/City_Weather.csv")
world_cities=world_cities.rename(columns={"Unnamed: 0":"City"})
world_cities

,City,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed
0,mallama,1.137396,-77.848901,51.19,95,75,3.98
1,santa barbara,-19.933333,-43.400000,55.65,93,20,1.03
2,mosquera,2.508610,-78.451102,78.19,85,100,14.32
3,pacora,5.527084,-75.459301,54.84,97,73,2.73
4,foso,5.700000,-1.283333,71.40,99,100,1.74
...,...,...,...,...,...,...,...
492,nakuru,-0.283333,36.066667,55.56,87,88,4.90
493,santo domingo,-0.250000,-79.150000,68.41,95,96,1.39
494,quito,-0.216667,-78.500000,50.02,94,75,3.44
495,puerto leguizamo,-0.193368,-74.781886,70.74,99,76,0.94


In [3]:
world_cities_humidity=world_cities[['Latitude','Longitude','Humidity']]
world_cities_humidity

,Latitude,Longitude,Humidity
0,1.137396,-77.848901,95
1,-19.933333,-43.400000,93
2,2.508610,-78.451102,85
3,5.527084,-75.459301,97
4,5.700000,-1.283333,99
...,...,...,...
492,-0.283333,36.066667,87
493,-0.250000,-79.150000,95
494,-0.216667,-78.500000,94
495,-0.193368,-74.781886,99


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
fig=gmaps.figure(map_type='HYBRID')
heatmap_layer=gmaps.heatmap_layer(world_cities_humidity[['Latitude','Longitude']],weights=world_cities_humidity['Humidity'])
fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
ideal_cities=world_cities.loc[(world_cities['Temperature'] > 70) & 
(world_cities['Temperature'] < 80) &
(world_cities['Humidity']<70) &
(world_cities['Wind Speed'] < 10) &
(world_cities['Cloudiness'] < 10)]
ideal_cities=ideal_cities.reset_index(drop=True)
ideal_cities


,City,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed
0,siverek,37.755025,39.316667,73.02,38,0,4.76
1,pataskala,39.995556,-82.674444,76.26,49,0,8.05
2,kholtoson,50.298100,103.309700,72.97,34,8,8.34
3,seryshevo,51.093333,128.378611,78.08,66,6,5.79
4,aksarka,66.560556,67.797500,72.81,60,3,6.60
5,lismore,-28.813280,153.275803,70.93,60,0,2.37
6,byron bay,-28.642010,153.611938,71.01,62,1,5.21
7,victoria point,-27.583290,153.311722,70.11,66,7,1.99
8,maryborough,-25.540730,152.704926,73.15,48,0,2.82
9,mage,-22.655556,-43.015278,73.49,66,2,3.60


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [24]:
hotel_names=[]
n=1
for i in range(0,12):
    try:
        lat=ideal_cities.loc[i,'Latitude']
        lon=ideal_cities.loc[i,'Longitude']
        url=f"https://maps.googleapis.com/maps/api/place/nearbysearch/json?&location={lat}%2C{lon}&radius=5000&type=lodging&key={g_key}"
        data=requests.get(url).json()
        hotel_name=data["results"][0]["name"]
        hotel_names.append(hotel_name)
        print(f"{n} {hotel_name}")
        n+=1
    except IndexError:
        print((f"{n} Index Error"))
        hotel_names.append("Index Error")
        n+=1



1 Mercan Hotel
2 James Helfrich apartments
3 Index Error
4 Sel'skiy Dom Kul'tuty
5 Index Error
6 Dawson Motor Inn
7 Backpackers Inn
8 Bay Retreat Motel
9 Blue Shades Motel
10 Hotel Vagalume
11 Hotel Pousada Pantal
12 Tucanos Park Hotel


In [29]:
hotel_df=ideal_cities
hotel_df["Hotel Name"]=hotel_names
hotel_df=hotel_df[hotel_df["Hotel Name"].str.contains("Index Error")==False]
hotel_df.to_csv("CSV/hotel_df.csv",index=False)
hotel_df

,City,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed,Hotel Name
0,siverek,37.755025,39.316667,73.02,38,0,4.76,Mercan Hotel
1,pataskala,39.995556,-82.674444,76.26,49,0,8.05,James Helfrich apartments
3,seryshevo,51.093333,128.378611,78.08,66,6,5.79,Sel'skiy Dom Kul'tuty
5,lismore,-28.813280,153.275803,70.93,60,0,2.37,Dawson Motor Inn
6,byron bay,-28.642010,153.611938,71.01,62,1,5.21,Backpackers Inn
7,victoria point,-27.583290,153.311722,70.11,66,7,1.99,Bay Retreat Motel
8,maryborough,-25.540730,152.704926,73.15,48,0,2.82,Blue Shades Motel
9,mage,-22.655556,-43.015278,73.49,66,2,3.60,Hotel Vagalume
10,pereira barreto,-20.633333,-51.116667,70.34,45,5,6.91,Hotel Pousada Pantal
11,ladario,-19.016667,-57.583333,71.58,57,8,2.93,Tucanos Park Hotel


In [30]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [31]:
# Add marker layer ontop of heat map
fig=gmaps.figure(map_type='HYBRID')
heatmap_layer=gmaps.heatmap_layer(world_cities_humidity[['Latitude','Longitude']],weights=world_cities_humidity['Humidity'])
fig.add_layer(heatmap_layer)
marker_layer=gmaps.marker_layer(locations=locations,info_box_content=hotel_info)
fig.add_layer(marker_layer)
fig

# Display figure


Figure(layout=FigureLayout(height='420px'))